# Finite field Nikodym problem

##2D

### Experiment 1: Giving AlphaEvolve access to a summary of a paper with a construction of size p^4 - p^3 + C * p^2 * log p, and asking it to reproduce it in code

Reference: Tamás Szőnyi, Antonello Cossidente, András Gács, Csaba Mengyán, Alessandro Siciliano, and Zsuzsa Weiner. On large minimal
blocking sets in PG(2, 𝑞). J. Comb. Des., 13(1):25–41, 2005.

**Prompt used**

Problem Statement:

The finite field Nikodym problem asks for the minimum size of a Nikodym set in a vector space over a finite field. Your goal is to find Nikodym sets in F_q^2 that are as small as possible, where q=p^2 for a given prime p.

A set N subseteq F_q^2 is a Nikodym set if, for every point x in F_q^2, there exists a line l passing through x such that the "punctured line" l setminus x is entirely contained within N. Formally, for every point x in F_q^2, there must exist a direction vector v in F_q^2 setminus 0 such that the set x + t cdot v : t in F_q, t neq 0 is a subset of N.

Your task is to write a Python function, search_for_best_construction, that directly returns a proposed Nikodym set.

The input to your function will be:

p: The prime number for the base field F_p. The full field is F_q where q=p^2.
d: The dimension of the vector space, which will be fixed at 2.

The function must return:

A NumPy array of shape (k, 2, 2) representing the k points in your proposed Nikodym set N. Each of the k points has 2 coordinates in F_q. Each coordinate from F_q is represented as a pair of integers from F_p.

How Your Function is Used:

Your function directly produces the set N. An external function will then verify if your returned set is a valid Nikodym set by checking the condition for every point x in the space F_q^2.

Your Goal and Evaluation:

Your goal is to define the set N in such a way that it has the smallest possible size (cardinality k) while still being a valid Nikodym set.

Your function will be evaluated based on the negative size of the resulting set N. A smaller set results in a less negative (i.e., higher) score. If your function returns an invalid Nikodym set, it will receive a very large penalty.

Hint: You must find a general construction that works well for various primes p. The previous construction you are seeing is not optimal yet -- a completely different construction can do much better! The optimal construction misses around p^3, or q^(3/2) points from the entire F_q^2.

An expert gave you the following advice on how to obtain this construction. Pay close attention to this advice:

How to Construct a Small Nikodym Set in a 2D Grid over a Special Number System

Objective

Our goal is to create a function that takes a prime number p and returns a large set of points (x, y) from a special p^2 x p^2 grid. This set has a magical property related to lines, making it a "Nikodym set." The key to building a small, efficient Nikodym set is to first build its complement (the set of points not in it) and then take everything else.

Conceptual Overview

We will work with a special number system called a "finite field," which we'll call F_q. Think of it as a grid of numbers where q = p^2. The construction has three main stages:

Build a "Unital" (U): This is a special p^3-point subset of our p^2 x p^2 grid, defined by a simple algebraic equation. It forms the backbone of our construction.

Build a "Blocking Set" (B): This is a carefully chosen subset of the Unital. It acts as a "patch" to ensure our final set has the required line properties.

Construct the Final Set (N): The points we want to exclude from our Nikodym set are the points in the Unital that are not in the Blocking Set (U \ B). Our final Nikodym set N is everything else in the grid.

Let's break down each stage into concrete programming steps.

Part 1: Setting up the Special Number System (F_q)

Before we can define points, we need to define the numbers that make up their coordinates. We'll be working in a field F_q where q = p^2. An easy way to think about these numbers is as 2D vectors (a, b) where a and b are integers from 0 to p-1. We'll represent a number x in F_q as x = a + b*α, where α is a special symbol.

Step 1.1: Create a Class or Structure for F_q Elements

You'll need a way to store these (a, b) pairs and define how they behave. A class is perfect for this.

In Python, this would be a class:
class FqElement:
    def init(self, a, b):
        self.a = a
        self.b = b
    # ... other methods will go here ...

Step 1.2: Find a "Non-Square" Number k

To define multiplication, we need a special number k in our base set (0, 1, ..., p-1). This number k must be a "non-quadratic residue," which means there's no integer z such that z*z % p == k.

How to find k: Loop through numbers i from 2 to p-1. For each i, calculate i^((p-1)/2) mod p. The first i for which this result is p-1 is your non-square k. For any prime p > 2, such a k is guaranteed to exist.

Step 1.3: Define Arithmetic for F_q Elements

Now, define the basic operations for two F_q elements, x = (a, b) and y = (c, d). Remember that all final calculations for a and b components are done modulo p.

Addition: x + y = ((a+c)%p, (b+d)%p)

Multiplication: This is the tricky one. It's based on the rule α*α = k.

(a + bα) * (c + dα) = ac + adα + bcα + bdα^2
= ac + (ad+bc)α + bdk
= (ac + bdk) + (ad+bc)*α

So, x * y = ((ac + bdk)%p, (ad + b*c)%p).

Step 1.4: Define Two Special Functions: Trace and Norm

These are two helper functions that map an F_q number back to the simpler numbers in F_p (i.e., numbers where the b component is 0). For an element x = (a, b):

Trace(x): This is x + x^p. In our (a, b) system, x^p is simply (a, -b).
Trace(x) = (a, b) + (a, -b) = (2a, 0).
Norm(x): This is x^(p+1) or x * x^p.
Norm(x) = (a, b) * (a, -b) = (aa - bb*k, 0).

Part 2: Building the Unital and the Blocking Set

Now that our number system is ready, we can start building the geometric sets.

Step 2.1: Generate All Points in the Grid

First, create a list or set of all possible points (x, y) in your p^2 x p^2 grid. This means both x and y can be any of the p^2 FqElement numbers you can create. There will be (p^2)^2 = p^4 points in total.

Step 2.2: Build the Unital (U)

The Unital is the set of all points (x, y) from your grid that satisfy the equation: Trace(x) + Norm(y) = 0.
Create an empty set, unital_U.
Loop through every point (x, y) in your full grid.
For each point, calculate result = Trace(x) + Norm(y).
Remember that Trace and Norm produce F_q elements of the form (val, 0). So the result will also be of this form. Check if the a-component of result is 0.
If result.a == 0, add the point (x, y) to your unital_U set.
After checking all p^4 points, unital_U will contain exactly p^3 points.

Step 2.3: Build the Special Y-coordinate Set (Y_special)

To build the Blocking Set, we first need to identify a special subset of F_q numbers that will serve as our y-coordinates. Let's call this Y_special.
Choose an integer r=2.
Get your alpha element, which is FqElement(0, 1).
Create an empty set, Y_special.
Loop u from 0 to p-1.
Inside that loop, loop v from 0 to p-1.
Calculate the special y-coordinate y = (u^r mod p) + valpha.
In code, this means creating an FqElement for u^r, which is FqElement(pow(u, r, p), 0).
And an FqElement for valpha, which is FqElement(0, v).
Then add them: y = FqElement(pow(u, r, p), v).
Add this y to your Y_special set.
This set will have p * (p+1)/2 unique elements.

Step 2.4: Build the Blocking Set (B)

The Blocking Set B is simply all the points in your Unital U whose y-coordinate comes from Y_special.
Create an empty set, blocking_set_B.
Loop through every point (x, y) in your unital_U set.
Check if the y-coordinate of the point, y, is present in your Y_special set.
If it is, add the point (x, y) to blocking_set_B.
This set will have p^2 * (p+1)/2 points.

Step 2.5: Modify the Blocking Set to Make It Proper

The goal is to strategically "puncture" the blocking set B you just made by removing a small number of points. This ensures no full lines are left inside it. This modified set is called B_proper in the paper.

Step 2.5.1: Define the "Puncturing" Structures

The paper's method for choosing which points to delete is based on two geometric objects:

A Cyclic Arc (A): This is a specific subset of q^2-q+1 points within the Unital U. Think of it as a special curve. This arc must contain the "point at infinity," which corresponds to (1:0:0) in the paper's coordinate system.

A Set of q Tangent Lines: Through the point at infinity, there are exactly q lines that are secants to the Unital U but are tangent to the arc A.

Implementation Note: The paper uses the existence of these structures to prove the construction works but doesn't provide a simple formula to generate them. For a practical implementation, you would need to define a specific cyclic arc A and then identify the lines tangent to it.

Step 2.5.2: Identify and Collect the Points to Delete

You will create a new set, points_to_delete. According to the paper, you need to remove two groups of points from your blocking_set_B:

Points on the Arc: All points where the arc A intersects with your blocking set B (except for the point at infinity itself).

Points on the Tangent Lines: All points in B that lie on any of the q special tangent lines identified above (again, excluding the point at infinity).

Loop through all points in blocking_set_B and add any that meet these criteria to your points_to_delete set.

Step 2.5.3: Create the Final Proper Blocking Set (B_proper)

Now, simply remove the collected points from your original blocking set.

In set terminology: B_proper = blocking_set_B - points_to_delete.

This new set, B_proper, is the proper blocking set B_proper described in the paper. The paper guarantees that this process removes enough points to break up any lines inside B but leaves enough points so that B_proper still successfully blocks all lines in the unital. This B_proper set will then have size approximately c * p^2 * log p.

Part 3: Final Assembly

You have all the components. Now, just put them together.

Step 3.1: Define the "Excluded Set" (S)

The set of points that are not in our final Nikodym set is S. This set is defined as all the points in the Unital except for the ones in the Blocking Set.
In set terminology, S = unital_U - blocking_set_B (set difference).

Step 3.2: Construct the Final Nikodym Set (N)

Your Nikodym set N is everything in the entire p^2 x p^2 grid except for the points in the excluded set S.
Get the set of all p^4 grid points from Step 2.1.
In set terminology, N = all_points - S.

Step 3.3: Format the Output

The final requirement is to return the set N as a specific type of array.
The output should be a NumPy array of shape (k, 2, 2), where k is the number of points in N.
Each of the k points is (x, y).
Each coordinate x or y is an FqElement represented by its (a, b) components.
So, a point (x, y) where x=(x_a, x_b) and y=(y_a, y_b) should be formatted as [[x_a, x_b], [y_a, y_b]].
Create a NumPy array of zeros with the correct shape and loop through your final set N, filling the array with the a and b components of each point's coordinates.
That's it! By following these steps, you will have successfully constructed a valid and very small Nikodym set.

The normalized scores you're seeing are simply the number of elements in the complement of the Nikodym set, you want this number to be as big as possible.

For p=29, the best possible normalized score you can get is about 24000. This is the construction you get by following the above steps, resulting in a blocking set of size c * p^2 * log p for some c. If you skip step 2.5 above, you get a normalized score of about 11700. A simplified version of step 2.5 can get you up to 19000, but we really want that 24000 score. You should try your best at implementing the difficult step 2.5 if possible.

Once you've reached a score of 24000-ish for p=29, you're on your own. How high can you push the score?

Good luck!

In [ ]:
#@title Evaluation code used

from typing import Tuple
import numpy as np
from numba import njit

@njit
def _mod_pow_numba(base: int, exp: int, mod: int) -> int:
  """A Numba-compatible implementation of modular exponentiation."""
  res = 1
  base %= mod
  while exp > 0:
    if exp % 2 == 1:
      res = (res * base) % mod
    base = (base * base) % mod
    exp //= 2
  return res


@njit
def _find_non_square_numba(p: int) -> int:
  """Finds a quadratic non-residue modulo p."""
  for w in range(2, p):
    # Use the Numba-compatible modular exponentiation function
    if _mod_pow_numba(w, (p - 1) // 2, p) == p - 1:
      return w
  return -1  # Should not be reached for p > 2


@njit
def _fq_add(
    a: Tuple[np.int64, np.int64], b: Tuple[np.int64, np.int64], p: int
) -> Tuple[np.int64, np.int64]:
  """Adds two elements in F_q, where q=p^2."""
  return (np.int64((a[0] + b[0]) % p), np.int64((a[1] + b[1]) % p))


@njit
def _fq_mul(
    a: Tuple[np.int64, np.int64],
    b: Tuple[np.int64, np.int64],
    p: int,
    w: int,
) -> Tuple[np.int64, np.int64]:
  """Multiplies two elements in F_q using irreducible polynomial x^2 - w."""
  # (a0 + a1*x) * (b0 + b1*x) = a0*b0 + (a0*b1 + a1*b0)*x + a1*b1*x^2
  # Since x^2 = w, this becomes: (a0*b0 + a1*b1*w) + (a0*b1 + a1*b0)*x
  res0 = (a[0] * b[0] + a[1] * b[1] * w) % p
  res1 = (a[0] * b[1] + a[1] * b[0]) % p
  return (np.int64(res0), np.int64(res1))


@njit
def _is_point_in_construction(
    point_to_check: np.ndarray, construction: np.ndarray
) -> bool:
  """Numba-friendly check if a point exists in the construction array."""
  # point_to_check has shape (2, 2) and construction has shape (k, 2, 2)
  for i in range(construction.shape[0]):
    if (
        construction[i, 0, 0] == point_to_check[0, 0]
        and construction[i, 0, 1] == point_to_check[0, 1]
        and construction[i, 1, 0] == point_to_check[1, 0]
        and construction[i, 1, 1] == point_to_check[1, 1]
    ):
      return True
  return False


@njit
def is_valid_nikodym_numba(construction: np.ndarray, p: int, d: int) -> bool:
  """Checks if a construction is a valid Nikodym set in F_q^d, where q=p^2."""
  if d != 2:
    return False

  w = _find_non_square_numba(p)
  if w == -1:
    return False  # Cannot define F_q

  # Create a set from the construction for efficient lookups.
  dummy_tuple = ((np.int64(0), np.int64(0)), (np.int64(0), np.int64(0)))
  construction_set = {dummy_tuple}
  construction_set.clear()
  for i in range(construction.shape[0]):
    p1 = (np.int64(construction[i, 0, 0]), np.int64(construction[i, 0, 1]))
    p2 = (np.int64(construction[i, 1, 0]), np.int64(construction[i, 1, 1]))
    construction_set.add((p1, p2))

  # Helper to check if a "punctured" line is fully contained in the set.
  # This function is defined inside to be captured by numba's jit compiler.
  def check_punctured_line_is_contained(point_x, direction_v, const_set):
    # Iterate through all non-zero elements t in F_q.
    for t_a in range(p):
      for t_b in range(p):
        if t_a == 0 and t_b == 0:
          continue

        t = (np.int64(t_a), np.int64(t_b))
        # Calculate the point on the line: x + t*v
        tv0 = _fq_mul(t, direction_v[0], p, w)
        tv1 = _fq_mul(t, direction_v[1], p, w)
        p_on_line0 = _fq_add(point_x[0], tv0, p)
        p_on_line1 = _fq_add(point_x[1], tv1, p)
        point_on_line = (p_on_line0, p_on_line1)

        if point_on_line not in const_set:
          return False
    return True

  # --- Main Loop: Iterate through EVERY point x in F_q^2 ---
  for x1a in range(p):
    for x1b in range(p):
      x1 = (np.int64(x1a), np.int64(x1b))
      for x2a in range(p):
        for x2b in range(p):
          x2 = (np.int64(x2a), np.int64(x2b))
          x = (x1, x2)
          found_line_for_x = False

          # A full set of directions in F_q^2 can be represented by
          # (1, m) for all m in F_q, and (0, 1).

          # Case 1: Directions v = (1, m)
          v1_fq = (np.int64(1), np.int64(0))
          for m_a in range(p):
            if found_line_for_x:
              break
            for m_b in range(p):
              m_fq = (np.int64(m_a), np.int64(m_b))
              v = (v1_fq, m_fq)
              if check_punctured_line_is_contained(x, v, construction_set):
                found_line_for_x = True
                break
          if found_line_for_x:
            continue

          # Case 2: Direction v = (0, 1)
          v0_fq = (np.int64(0), np.int64(0))
          v = (v0_fq, v1_fq)
          if check_punctured_line_is_contained(x, v, construction_set):
            found_line_for_x = True

          if not found_line_for_x:
            return False  # Not a valid Nikodym set.

  return True


def calculate_score(construction: np.ndarray, p: int, d: int) -> float:
  """Calculates the score for a given construction."""
  if (
      not isinstance(construction, np.ndarray)
      or construction.ndim != 3
      or construction.shape[1] != d
      or construction.shape[2] != 2
  ):
    return np.inf

  if construction.shape[0] == 0:
    return np.inf

  # Remove duplicate points before scoring
  unique_tuples = {tuple(tuple(coord) for coord in row) for row in construction}
  unique_construction = np.array(list(unique_tuples), dtype=np.int64)
  size = unique_construction.shape[0]

  # NOTE: The validation can be very slow (O(p^8)) and is best suited for
  # small primes. For larger primes, the correctness of the algebraic
  # construction is assumed.
  is_nikodym = is_valid_nikodym_numba(unique_construction, p, d)

  if not is_nikodym:
    return np.inf  # Heavily penalize invalid sets

  return -float(size)


def evaluate(params: int) -> tuple[dict[str, float], dict[str, str]]:
  """Evaluates a Nikodym set construction for given parameters."""
  result = {}
  del params  # Not used directly

  # Use smaller primes as the O(p^8) validation is computationally intensive.
  primes_to_test = [5, 7, 11, 29]
  dims_to_test = [2]

  scores = []
  best_constructions = {}
  time_start = time.time()
  for d in dims_to_test:
    for p in primes_to_test:
      q = p * p
      construction = search_for_best_construction(p, d)
      score = calculate_score(construction, p, d) + np.random.uniform(0, 0.0001)

      if score != np.inf and p > 20:
        neg_size = -score
        # Normalize score by the size of the space (q^2).
        normalized_score = neg_size - q**d
        scores.append(normalized_score)
        logging.info(
            'p=%d, q=%d, d=%d: |N|=%.0f, normalized_score (density)=%.4f',
            p,
            q,
            d,
            neg_size,
            normalized_score,
        )

        # best_constructions[f'best_construction_p{p}_d{d}'] = construction
        # best_constructions[f'best_score_p{p}_d{d}'] = neg_size
        best_constructions[f'normalized_score_p{p}'] = -normalized_score
      else:
        print(f'p={p}, d={d}: Invalid construction found.')
        # scores.append(0)  # Penalize failure

  average_score = (
      np.mean(scores) + np.random.uniform(0, 0.0001) if scores else 1000
  )
  logging.info('Time taken for full evaluation: %s', time.time() - time_start)
  logging.info('Average score (density): %s', average_score)
  result['score'] = -average_score

  return result, best_constructions


First, AlphaEvolve found a simple construction giving a set of size p^4 - p^3/2:

In [ ]:
#@title A simple construction of size p^4 - p^3/2, ignoring the difficult steps (2.5) in the prompt


def search_for_best_construction(p: int, d: int):
  """Search for the best Nikodym set for F_q^2."""

  if d != 2:
    # Fallback for other dimensions to prevent errors.
    return np.array([], dtype=np.int64).reshape(0, d, 2)

  # Step 1.2: Find a non-square number k
  k = -1
  for i in range(1, p):
    is_square = False
    for j in range(p):
      if (j * j) % p == i:
        is_square = True
        break
    if not is_square:
      k = i
      break
  if k == -1: # Should not happen for p > 2
      k = 2 if p == 3 else 3 # Fallback for p=2 (not used) or small primes
      for i in range(2, p):
          if pow(i, (p - 1) // 2, p) == p - 1:
              k = i
              break

  # Step 1.1, 1.3, 1.4: Create a Class for F_q Elements with arithmetic
  class FqElement:
    def __init__(self, a, b):
      self.a = a % p
      self.b = b % p
      self.p = p
      self.k = k

    def __add__(self, other):
      return FqElement(self.a + other.a, self.b + other.b)

    def __sub__(self, other):
      return FqElement(self.a - other.a, self.b - other.b)

    def __mul__(self, other):
      a, b = self.a, self.b
      c, d = other.a, other.b
      return FqElement(a * c + b * d * self.k, a * d + b * c)

    def __eq__(self, other):
      return self.a == other.a and self.b == other.b

    def __hash__(self):
      return hash((self.a, self.b))

    def __repr__(self):
      return f"({self.a}+{self.b}α)"

    def is_zero(self):
      return self.a == 0 and self.b == 0

  def trace(x: FqElement) -> FqElement:
    return FqElement(2 * x.a, 0)

  def norm(y: FqElement) -> FqElement:
    a, b = y.a, y.b
    return FqElement(a * a - b * b * k, 0)

  # Step 2.1: Generate all points in the grid
  all_fq_elements = [FqElement(a, b) for a in range(p) for b in range(p)]
  all_points = list(itertools.product(all_fq_elements, repeat=2))

  # Step 2.2: Build the Unital (U)
  unital_U = set()
  for x, y in all_points:
    if (trace(x) + norm(y)).is_zero():
      unital_U.add((x, y))

  # Step 2.3: Build the Special Y-coordinate Set (Y_special)
  Y_special = set()
  r = 2
  alpha = FqElement(0, 1)
  for u in range(p):
    for v in range(p):
      term1 = FqElement(pow(u, r, p), 0)
      term2 = FqElement(0, v) # v*alpha
      Y_special.add(term1 + term2)

  # Step 2.4: Build the Blocking Set (B)
  blocking_set_B = set()
  for x, y in unital_U:
    if y in Y_special:
      blocking_set_B.add((x, y))

  # Step 3.1: Define the "Excluded Set" (S)
  excluded_set_S = unital_U - blocking_set_B

  # Step 3.2: Construct the Final Nikodym Set (N)
  all_points_set = set(all_points)
  nikodym_set_N = all_points_set - excluded_set_S

  # Step 3.3: Format the output
  k_size = len(nikodym_set_N)
  best_construction = np.zeros((k_size, 2, 2), dtype=np.int64)
  for i, (x, y) in enumerate(nikodym_set_N):
    best_construction[i, 0, 0] = x.a
    best_construction[i, 0, 1] = x.b
    best_construction[i, 1, 0] = y.a
    best_construction[i, 1, 1] = y.b

  return best_construction


Next, AlphaEvolve added a simplified version of step 2.5, that improved the size of the set to p^4 - (3/4) p^3

In [ ]:
#@title A construction of size ~p^4 - (3/4) p^3, with a simple, suboptimal blocking set

"""AlphaEvolve experiment for the Nikodym problem in F_q^2."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from typing import Any, Callable, Mapping, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
from scipy.optimize import milp, LinearConstraint, Bounds
from itertools import product

njit = numba.njit

# Helper functions and classes for F_q arithmetic.

def find_non_square(p):
    """Finds a non-quadratic residue modulo p."""
    for k in range(2, p):
        if pow(k, (p - 1) // 2, p) == p - 1:
            return k
    return -1 # Should not happen for p > 2

class FqElement:
    """Represents an element in F_q where q = p^2."""
    p = -1
    k = -1

    def __init__(self, a, b):
        self.a = a % self.p
        self.b = b % self.p

    def __add__(self, other):
        return FqElement(self.a + other.a, self.b + other.b)

    def __sub__(self, other):
        return FqElement(self.a - other.a, self.b - other.b)

    def __mul__(self, other):
        # (a + b*alpha) * (c + d*alpha) = ac + (ad+bc)*alpha + bd*alpha^2
        # = (ac + bdk) + (ad+bc)*alpha
        a, b = self.a, self.b
        c, d = other.a, other.b
        return FqElement(a * c + b * d * self.k, a * d + b * c)

    def __eq__(self, other):
        return self.a == other.a and self.b == other.b

    def __hash__(self):
        return hash((self.a, self.b))

    def __repr__(self):
        return f"({self.a},{self.b})"

    def is_zero(self):
        return self.a == 0 and self.b == 0

def trace(x: FqElement):
    """Computes the trace of an F_q element."""
    # Trace(x) = x + x^p = (a + b*alpha) + (a - b*alpha) = 2a
    return FqElement(2 * x.a, 0)

def norm(x: FqElement):
    """Computes the norm of an F_q element."""
    # Norm(x) = x * x^p = (a + b*alpha) * (a - b*alpha) = a^2 - b^2*k
    return FqElement(x.a**2 - x.b**2 * FqElement.k, 0)


# Here are the best constructions for small values of the parameter,
# that you have found so far:

# PREVIOUS CONSTRUCTIONS START HERE


normalized_score_p29_iqhd = 17864.000064290594


# PREVIOUS CONSTRUCTIONS END HERE


def search_for_best_construction(p: int, d: int):
    """Search for the best Nikodym set for F_q^2."""
    if d != 2:
        return np.array([], dtype=np.int64).reshape(0, d, 2)

    # Part 1: Setting up the Special Number System (F_q)
    FqElement.p = p
    FqElement.k = find_non_square(p)

    fq_elements = [FqElement(a, b) for a in range(p) for b in range(p)]
    fq_elements_set = set(fq_elements)

    # Part 2: Building the Unital and the Blocking Set
    # Step 2.1: Generate All Points in the Grid
    all_points_in_fq2 = list(itertools.product(fq_elements, repeat=2))

    # Step 2.2: Build the Unital (U)
    unital_U = set()
    for x, y in all_points_in_fq2:
        if (trace(x) + norm(y)).is_zero():
            unital_U.add((x, y))

    # Step 2.3: Build the Special Y-coordinate Set (Y_special)
    alpha = FqElement(0, 1)
    r = 2
    Y_special = set()
    for u in range(p):
        u_r_term = FqElement(pow(u, r, p), 0)
        for v in range(p):
            v_alpha_term = FqElement(0, v)
            y = u_r_term + v_alpha_term
            Y_special.add(y)

    # Step 2.4: Build the Blocking Set (B)
    blocking_set_B = set()
    for x, y in unital_U:
        if y in Y_special:
            blocking_set_B.add((x, y))

    # Step 2.5: Modify the Blocking Set (simplified version)
    proper_blocking_set_B = set()
    p_minus_1_div_2 = (p - 1) // 2
    for x, y in blocking_set_B:
        # Keep point if x.a is a square in F_p
        if x.a == 0 or pow(x.a, p_minus_1_div_2, p) == 1:
            proper_blocking_set_B.add((x, y))

    # Part 3: Final Assembly
    # Step 3.1: Define the "Excluded Set" (S)
    # S = U \ B_proper
    excluded_set_S = unital_U - proper_blocking_set_B

    # Step 3.2: Construct the Final Nikodym Set (N)
    # N = all_points - S
    all_points_set = set(all_points_in_fq2)
    nikodym_set_N = all_points_set - excluded_set_S

    # Step 3.3: Format the Output
    k = len(nikodym_set_N)
    result_array = np.zeros((k, 2, 2), dtype=np.int64)
    for i, (x, y) in enumerate(nikodym_set_N):
        result_array[i, 0, 0] = x.a
        result_array[i, 0, 1] = x.b
        result_array[i, 1, 0] = y.a
        result_array[i, 1, 1] = y.b

    return result_array




This is where the difficulties started. AlphaEvolve implemented several versions of step 2.5 (step 2 in the paper), but was confused that none of them had error term p^2 log p. Then after a few hours AlphaEvolve realized it can get a p^4 - p^3 + O(p^2 log p) sized construction by ignoring the part of the prompt about section 4.3 and instead building a blocking set greedily, in each step it adds the single point that covers the greatest number of still-uncovered secant lines. (This was explained in the paper as a good solution, but it didn't appear in the prompt.)

In [ ]:
#@title Final code found by AlphaEvolve

"""AlphaEvolve experiment for the Nikodym problem in F_q^2."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from typing import Any, Callable, Mapping, List, Tuple
import scipy.linalg as la
import collections
import copy
import collections
import math
import numba
from scipy.optimize import milp, LinearConstraint, Bounds
from itertools import product

njit = numba.njit


# Here are the best constructions for small values of the parameter,
# that you have found so far:

# PREVIOUS CONSTRUCTIONS START HERE


normalized_score_p29_iqhd = 20736.000080043683


# PREVIOUS CONSTRUCTIONS END HERE


_FQ_P = -1
_FQ_K = -1


class FqElement:
    """Represents an element of F_q where q=p^2."""

    def __init__(self, a, b):
        self.a = a % _FQ_P
        self.b = b % _FQ_P

    @staticmethod
    def initialize(p):
        global _FQ_P, _FQ_K
        if _FQ_P == p:
            return
        _FQ_P = p
        if p == 2:
            _FQ_K = -1  # Sentinel for p=2 not supported by this construction
            return

        for i in range(2, p):
            if pow(i, (p - 1) // 2, p) == p - 1:
                _FQ_K = i
                return

    def __add__(self, other):
        return FqElement(self.a + other.a, self.b + other.b)

    def __sub__(self, other):
        return FqElement(self.a - other.a, self.b - other.b)

    def __mul__(self, other):
        p, k = _FQ_P, _FQ_K
        a, b = self.a, self.b
        c, d = other.a, other.b
        res_a = (a * c + b * d * k) % p
        res_b = (a * d + b * c) % p
        return FqElement(res_a, res_b)

    def __eq__(self, other):
        if not isinstance(other, FqElement):
            return NotImplemented
        return self.a == other.a and self.b == other.b

    def __hash__(self):
        return hash((self.a, self.b))

    def __repr__(self):
        return f"({self.a},{self.b})_alpha"

    def trace(self):
        return FqElement((2 * self.a) % _FQ_P, 0)

    def norm(self):
        p, k = _FQ_P, _FQ_K
        a, b = self.a, self.b
        res_a = (a * a - b * b * k) % p
        return FqElement(res_a, 0)


def search_for_best_construction(p: int, d: int):
  """Search for the best Nikodym set for F_q^2."""
  if d != 2:
    return np.array([], dtype=np.int64).reshape(0, d, 2)

  FqElement.initialize(p)
  if _FQ_K == -1:  # Handles p=2, where construction is not supported
    fq_elements = list(itertools.product(range(p), repeat=2))
    all_points_in_fq2 = list(itertools.product(fq_elements, repeat=2))
    return np.array(all_points_in_fq2, dtype=np.int64)

  # Part 2.2: Build the Unital (U)
  unital_U = set()
  k = _FQ_K
  all_points_coords = itertools.product(range(p), repeat=4)
  for x_a, x_b, y_a, y_b in all_points_coords:
    if (2 * x_a + y_a * y_a - y_b * y_b * k) % p == 0:
      x = FqElement(x_a, x_b)
      y = FqElement(y_a, y_b)
      unital_U.add((x, y))

  # Part 2.3 & 2.4: Build the Blocking Set (B) from points in U
  # We take B to be the entire unital U. This simplifies a choice point and
  # forces B_proper to be a blocking set for all secants of U.
  # The greedy set-cover algorithm will then find a small blocking set of U.
  blocking_set_B = unital_U

  # Step 2.5 (Advanced): Reduce B to a smaller, proper blocking set B_proper
  # using a greedy set-cover algorithm.
  q = p * p
  all_fq_elements = [FqElement(a, b) for a in range(p) for b in range(p)]
  one = FqElement(1, 0)
  zero = FqElement(0, 0)
  directions = [(one, m) for m in all_fq_elements]
  directions.append((zero, one))

  # Pre-computation: Find all secant lines and their incidences with points in B.
  secant_B_map = collections.defaultdict(list)
  point_to_secants = collections.defaultdict(list)

  for dir_idx, (v_x, v_y) in enumerate(directions):
    partitions_U = collections.defaultdict(list)
    for point_U in unital_U:
      x, y = point_U
      invariant = v_y * x - v_x * y
      partitions_U[invariant].append(point_U)

    for inv, line_U in partitions_U.items():
      if len(line_U) == p + 1:  # This is a secant line of U
        secant_id = (dir_idx, inv)
        secant_B_map[secant_id] = line_U
        for point in line_U:
            point_to_secants[point].append(secant_id)

  uncovered_secants = set(secant_B_map.keys())
  blocking_set_B_proper = set()
  B_candidates = blocking_set_B

  # Count how many currently uncovered secants each candidate point covers.
  # Initially, all secants are uncovered.
  point_uncovered_counts = {p_cand: len(point_to_secants[p_cand])
                            for p_cand in B_candidates}
  point_uncovered_counts = collections.defaultdict(int, point_uncovered_counts)

  # Greedy loop: iteratively add the point that covers the most uncovered secants.
  while uncovered_secants:
    if not point_uncovered_counts:
      break

    best_point = max(point_uncovered_counts, key=point_uncovered_counts.get)

    if point_uncovered_counts[best_point] <= 0:
      break

    blocking_set_B_proper.add(best_point)
    del point_uncovered_counts[best_point]

    secants_just_covered = []
    for secant_id in point_to_secants[best_point]:
      if secant_id in uncovered_secants:
        uncovered_secants.remove(secant_id)
        secants_just_covered.append(secant_id)

    # Update counts for neighboring points on the just-covered secants.
    for secant_id in secants_just_covered:
      for p_neighbor in secant_B_map[secant_id]:
        if p_neighbor in point_uncovered_counts:
          point_uncovered_counts[p_neighbor] -= 1

  # Part 3.2: Construct the final Nikodym set N
  # The set of excluded points is S = U \ B_proper.
  # The Nikodym set is N = F_q^2 \ S.
  excluded_set_S = unital_U - blocking_set_B_proper
  nikodym_set_N = set()
  all_points_coords = itertools.product(range(p), repeat=4)
  for x_a, x_b, y_a, y_b in all_points_coords:
      x = FqElement(x_a, x_b)
      y = FqElement(y_a, y_b)
      if (x, y) not in excluded_set_S:
          nikodym_set_N.add((x, y))

  # Part 3.3: Format the output
  output_list = []
  for x, y in nikodym_set_N:
      output_list.append([[x.a, x.b], [y.a, y.b]])
  return np.array(output_list, dtype=np.int64)



### Experiment 2: only telling AlphaEvolve that a value of p^4 - p^3 + C * p^2 * log p is possible, but not telling it how

**Prompt used**

Problem Statement:

The finite field Nikodym problem asks for the minimum size of a Nikodym set in a vector space over a finite field. Your goal is to find Nikodym sets in F_q^2 that are as small as possible, where q=p^2 for a given prime p.

A set N subseteq F_q^2 is a Nikodym set if, for every point x in F_q^2, there exists a line l passing through x such that the "punctured line" l setminus x is entirely contained within N. Formally, for every point x in F_q^2, there must exist a direction vector v in F_q^2 setminus 0 such that the set x + t cdot v : t in F_q, t neq 0 is a subset of N.

Your task is to write a Python function, search_for_best_construction, that directly returns a proposed Nikodym set.

The input to your function will be:

p: The prime number for the base field F_p. The full field is F_q where q=p^2.
d: The dimension of the vector space, which will be fixed at 2.

The function must return:

A NumPy array of shape (k, 2, 2) representing the k points in your proposed Nikodym set N. Each of the k points has 2 coordinates in F_q. Each coordinate from F_q is represented as a pair of integers from F_p.

How Your Function is Used:

Your function directly produces the set N. An external function will then verify if your returned set is a valid Nikodym set by checking the condition for every point x in the space F_q^2.

Your Goal and Evaluation:

Your goal is to define the set N in such a way that it has the smallest possible size (cardinality k) while still being a valid Nikodym set.

Your function will be evaluated based on the negative size of the resulting set N. A smaller set results in a less negative (i.e., higher) score. If your function returns an invalid Nikodym set, it will receive a very large penalty.

Hint: You must find a general construction that works well for various primes p. The optimal construction misses around p^3 - C * p^2 * log p points from the entire (F_q)^2, meaning the Nikodym set has size p^4 - p^3 + C * p^2 * log p. This is what you should aim for. First try to find a construction of size p^4 - c * p^3, and once you have this, try to go for the optimal value of p^4 - p^3 + C * p^2 * log p. The "normalized_score" you are seeing for p=29 is the size of the complement. Once you are on the right track and the complement will be cubic, this score should be above 10000 (or very close to it). The optimal construction of size p^4 - p^3 + C * p^2 * log p will have a score of above 20000 for p=29.

Good luck!

AlphaEvolve finds a construction, that Deep Think believes has size p^4 - p^3 + C p^2 log p.

In [ ]:
#@title Code found by AlphaEvolve

"""AlphaEvolve experiment for the Nikodym problem in F_q^2."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
from scipy.optimize import milp, LinearConstraint, Bounds
from itertools import product

njit = numba.njit


def get_prime_factorization(num: int) -> Mapping[int, int]:
    """Returns the prime factorization of num as a dict."""
    factors = collections.defaultdict(int)
    d = 2
    temp = num
    while d * d <= temp:
        while temp % d == 0:
            factors[d] += 1
            temp //= d
        d += 1
    if temp > 1:
        factors[temp] += 1
    return factors


# Here are the best constructions for small values of the parameter,
# that you have found so far:

# PREVIOUS CONSTRUCTIONS START HERE


normalized_score_p29_iqhd = 19285.00002998754


# PREVIOUS CONSTRUCTIONS END HERE


def search_for_best_construction(p: int, d: int):
  """Search for the best Nikodym set for F_q^2."""

  if d != 2:
    # Fallback for other dimensions to prevent errors.
    return np.array([], dtype=np.int64).reshape(0, d, 2)






  # Find coefficients for an irreducible polynomial x^2 = irr_a*x + irr_b over F_p
  # to construct F_q.
  # For p>2, x^2 - k is irreducible for k non-square. (a=0, b=k)
  # For p=2, x^2 + x + 1 is irreducible. (a=1, b=1 mod 2, from x^2 = -x-1)
  if p == 2:
      irr_a, irr_b = 1, 1
  else:
      non_square = next(i for i in range(1, p) if pow(i, (p - 1) // 2, p) == p - 1)
      irr_a, irr_b = 0, non_square
  q = p * p

  # This construction refines Guo, Kopparty, and Sudan (2013).
  # The complement set is M' = {(x,y) | Tr(x)=N(y) and y not in S},
  # for a special set S. The resulting Nikodym set N' = F_q^2 \ M' has size
  # p^4 - (p^3 - p*|S|) = p^4 - p^3 + p*|S|.
  # The score is based on the size of the complement, p^3 - p*|S|.
  # The hints suggest that this value should be aimed for, and that a baseline
  # of p^3 is not optimal.

  # F_q arithmetic functions (elements are (a,b) for a+b*w where w^2=irr_a*w+irr_b)
  def fq_mul(a, b):
      return ((a[0] * b[0] + irr_b * a[1] * b[1]) % p,
              (a[0] * b[1] + a[1] * b[0] + irr_a * a[1] * b[1]) % p)

  def fq_pow(a, n):
      res = (1, 0)
      base = a
      while n > 0:
          if n % 2 == 1:
              res = fq_mul(res, base)
          base = fq_mul(base, base)
          n //= 2
      return res

  # Find a generator for F_q^*
  q_minus_1 = q - 1
  prime_factors = get_prime_factorization(q_minus_1)

  random.seed(p)
  while True:
      g = (random.randint(0, p - 1), random.randint(0, p - 1))
      if g == (0, 0):
          continue
      is_generator = True
      for p_factor in prime_factors:
          if fq_pow(g, q_minus_1 // p_factor) == (1, 0):
              is_generator = False
              break
      if is_generator:
          break

  # Construct the set S of "special" points.
  # Theory suggests |S| should be large enough to be a hitting set for a
  # family of p-dimensional subspaces. A union bound argument on a simplified
  # model suggests m > p * log((q-1)/2). We use a value slightly above this
  # bound to ensure the construction is valid. A smaller m yields a better score.
  m_bound = p * (math.log(q - 1) - math.log(2))
  m = math.ceil(m_bound)  # Use the smallest integer satisfying the bound.
  m = int(min(m, q - 1))
  S = set()
  current_g_power = (1, 0)  # Start with g^0=1
  for _ in range(m):
      current_g_power = fq_mul(current_g_power, g)  # g^1, g^2, ...
      S.add(current_g_power)

  # Vectorized implementation of the construction.
  coords = np.mgrid[0:p, 0:p, 0:p, 0:p]
  all_points_flat = coords.reshape(4, -1).T.astype(np.int64)
  x1, x2, y1, y2 = all_points_flat.T

  # Condition for being in the base complement set M: Tr(x) = N(y)
  if p == 2:
      # For F_4 = F_2(w) with w^2+w+1=0.
      # x = x1+x2*w, Tr(x) = x2
      # y = y1+y2*w, N(y) = y1^2+y1*y2+y2^2
      trace_x = x2
      norm_y = (y1**2 + y1 * y2 + y2**2) % p
  else:
      # For F_{p^2} = F_p(sqrt(k))
      # x = x1+x2*sqrt(k), Tr(x) = 2*x1
      # y = y1+y2*sqrt(k), N(y) = y1^2-k*y2^2
      trace_x = (2 * x1) % p
      norm_y = (y1**2 - irr_b * y2**2) % p
  in_m_basic = (trace_x == norm_y)

  # The refined complement M' requires y to not be in S.
  S_mask = np.zeros((p, p), dtype=bool)
  for s_val in S:
      S_mask[s_val[0], s_val[1]] = True
  y_not_in_S = ~S_mask[y1, y2]

  in_m_prime = in_m_basic & y_not_in_S

  # The Nikodym set N' contains all points not in M'.
  nikodym_points_flat = all_points_flat[~in_m_prime]

  return nikodym_points_flat.reshape(-1, 2, 2)



In [ ]:
#@title Initial program used

def search_for_best_construction(p: int, d: int):
  """Search for the best Nikodym set for F_q^2."""

  if d == 2:
    # 1. First, create all elements of the field F_q, where q=p^2.
    #    Each element is a tuple of two integers from F_p.
    fq_elements = list(itertools.product(range(p), repeat=2))

    # 2. Now, create all points in the space F_q^2.
    #    Each point is a pair of F_q elements.
    all_points_in_fq2 = list(itertools.product(fq_elements, repeat=2))

    # This creates a NumPy array with the correct shape (p**4, 2, 2).
    best_construction = np.array(all_points_in_fq2, dtype=np.int64)
  else:
    # Fallback for other dimensions to prevent errors.
    best_construction = np.array([], dtype=np.int64).reshape(0, d, 2)

  return best_construction

In [ ]:
#@title Verification code


@njit
def _mod_pow_numba(base: int, exp: int, mod: int) -> int:
  """A Numba-compatible implementation of modular exponentiation."""
  res = 1
  base %= mod
  while exp > 0:
    if exp % 2 == 1:
      res = (res * base) % mod
    base = (base * base) % mod
    exp //= 2
  return res


@njit
def _find_non_square_numba(p: int) -> int:
  """Finds a quadratic non-residue modulo p."""
  for w in range(2, p):
    # Use the Numba-compatible modular exponentiation function
    if _mod_pow_numba(w, (p - 1) // 2, p) == p - 1:
      return w
  return -1  # Should not be reached for p > 2


@njit
def _fq_add(
    a: Tuple[np.int64, np.int64], b: Tuple[np.int64, np.int64], p: int
) -> Tuple[np.int64, np.int64]:
  """Adds two elements in F_q, where q=p^2."""
  return (np.int64((a[0] + b[0]) % p), np.int64((a[1] + b[1]) % p))


@njit
def _fq_mul(
    a: Tuple[np.int64, np.int64],
    b: Tuple[np.int64, np.int64],
    p: int,
    w: int,
) -> Tuple[np.int64, np.int64]:
  """Multiplies two elements in F_q using irreducible polynomial x^2 - w."""
  # (a0 + a1*x) * (b0 + b1*x) = a0*b0 + (a0*b1 + a1*b0)*x + a1*b1*x^2
  # Since x^2 = w, this becomes: (a0*b0 + a1*b1*w) + (a0*b1 + a1*b0)*x
  res0 = (a[0] * b[0] + a[1] * b[1] * w) % p
  res1 = (a[0] * b[1] + a[1] * b[0]) % p
  return (np.int64(res0), np.int64(res1))


@njit
def _is_point_in_construction(
    point_to_check: np.ndarray, construction: np.ndarray
) -> bool:
  """Numba-friendly check if a point exists in the construction array."""
  # point_to_check has shape (2, 2) and construction has shape (k, 2, 2)
  for i in range(construction.shape[0]):
    if (
        construction[i, 0, 0] == point_to_check[0, 0]
        and construction[i, 0, 1] == point_to_check[0, 1]
        and construction[i, 1, 0] == point_to_check[1, 0]
        and construction[i, 1, 1] == point_to_check[1, 1]
    ):
      return True
  return False


@njit
def is_valid_nikodym_numba(construction: np.ndarray, p: int, d: int) -> bool:
  """Checks if a construction is a valid Nikodym set in F_q^d, where q=p^2."""
  if d != 2:
    return False

  w = _find_non_square_numba(p)
  if w == -1:
    return False  # Cannot define F_q

  # Create a set from the construction for efficient lookups.
  dummy_tuple = ((np.int64(0), np.int64(0)), (np.int64(0), np.int64(0)))
  construction_set = {dummy_tuple}
  construction_set.clear()
  for i in range(construction.shape[0]):
    p1 = (np.int64(construction[i, 0, 0]), np.int64(construction[i, 0, 1]))
    p2 = (np.int64(construction[i, 1, 0]), np.int64(construction[i, 1, 1]))
    construction_set.add((p1, p2))

  # Helper to check if a "punctured" line is fully contained in the set.
  # This function is defined inside to be captured by numba's jit compiler.
  def check_punctured_line_is_contained(point_x, direction_v, const_set):
    # Iterate through all non-zero elements t in F_q.
    for t_a in range(p):
      for t_b in range(p):
        if t_a == 0 and t_b == 0:
          continue

        t = (np.int64(t_a), np.int64(t_b))
        # Calculate the point on the line: x + t*v
        tv0 = _fq_mul(t, direction_v[0], p, w)
        tv1 = _fq_mul(t, direction_v[1], p, w)
        p_on_line0 = _fq_add(point_x[0], tv0, p)
        p_on_line1 = _fq_add(point_x[1], tv1, p)
        point_on_line = (p_on_line0, p_on_line1)

        if point_on_line not in const_set:
          return False
    return True

  # --- Main Loop: Iterate through EVERY point x in F_q^2 ---
  for x1a in range(p):
    for x1b in range(p):
      x1 = (np.int64(x1a), np.int64(x1b))
      for x2a in range(p):
        for x2b in range(p):
          x2 = (np.int64(x2a), np.int64(x2b))
          x = (x1, x2)
          found_line_for_x = False

          # A full set of directions in F_q^2 can be represented by
          # (1, m) for all m in F_q, and (0, 1).

          # Case 1: Directions v = (1, m)
          v1_fq = (np.int64(1), np.int64(0))
          for m_a in range(p):
            if found_line_for_x:
              break
            for m_b in range(p):
              m_fq = (np.int64(m_a), np.int64(m_b))
              v = (v1_fq, m_fq)
              if check_punctured_line_is_contained(x, v, construction_set):
                found_line_for_x = True
                break
          if found_line_for_x:
            continue

          # Case 2: Direction v = (0, 1)
          v0_fq = (np.int64(0), np.int64(0))
          v = (v0_fq, v1_fq)
          if check_punctured_line_is_contained(x, v, construction_set):
            found_line_for_x = True

          if not found_line_for_x:
            return False  # Not a valid Nikodym set.

  return True


def calculate_score(construction: np.ndarray, p: int, d: int) -> float:
  """Calculates the score for a given construction."""
  if (
      not isinstance(construction, np.ndarray)
      or construction.ndim != 3
      or construction.shape[1] != d
      or construction.shape[2] != 2
  ):
    return np.inf

  if construction.shape[0] == 0:
    return np.inf

  # Remove duplicate points before scoring
  unique_tuples = {tuple(tuple(coord) for coord in row) for row in construction}
  unique_construction = np.array(list(unique_tuples), dtype=np.int64)
  size = unique_construction.shape[0]

  # NOTE: The validation can be very slow (O(p^8)) and is best suited for
  # small primes. For larger primes, the correctness of the algebraic
  # construction is assumed.
  is_nikodym = is_valid_nikodym_numba(unique_construction, p, d)

  if not is_nikodym:
    return np.inf  # Heavily penalize invalid sets

  return -float(size)


def evaluate() -> tuple[dict[str, float], dict[str, str]]:
  """Evaluates a Nikodym set construction for given parameters."""
  result = {}

  # Use smaller primes as the O(p^8) validation is computationally intensive.
  primes_to_test = [5, 7, 11, 29]
  dims_to_test = [2]

  scores = []
  best_constructions = {}
  time_start = time.time()
  for d in dims_to_test:
    for p in primes_to_test:
      q = p * p
      construction = search_for_best_construction(p, d)
      score = calculate_score(construction, p, d) + np.random.uniform(0, 0.0001)

      if score != np.inf and p > 20:
        neg_size = -score
        # Normalize score by the size of the space (q^2).
        normalized_score = neg_size - q**d
        scores.append(normalized_score)
        logging.info(
            'p=%d, q=%d, d=%d: |N|=%.0f, normalized_score (density)=%.4f',
            p,
            q,
            d,
            neg_size,
            normalized_score,
        )

      else:
        print(f'p={p}, d={d}: Invalid construction found.')
        # scores.append(0)  # Penalize failure

  average_score = (
      np.mean(scores) + np.random.uniform(0, 0.0001) if scores else 1000
  )
  logging.info('Time taken for full evaluation: %s', time.time() - time_start)
  logging.info('Average score (density): %s', average_score)
  result['score'] = -average_score

  return result

## 3D

**Prompt used**

The finite field Nikodym problem asks for the minimum size of a Nikodym set in the vector space F_p^d. Your goal is to find Nikodym sets in F_p^3 that are as small as possible.

A set N subseteq F_p^3 is a Nikodym set if, for every point x in F_p^3, there exists a line l passing through x such that the "punctured line" l setminus x is entirely contained within N. Formally, for every point x in F_p^3, there must exist a direction vector v in F_p^3 setminus 0 such that the set x + (t cdot v) : t in F_p,t neq 0 is a subset of N.

Instead of defining the set of points with a specific mapping, your task is to write a Python function, search_for_best_construction, that directly returns a proposed Nikodym set.

The input to your function will be:

p: The prime number for the finite field F_p. It will always be an odd prime, strictly greater than 20.

d: The dimension of the vector space, which will be fixed at 3.

The function must return:

A NumPy array of shape (k, 3) representing the k points in your proposed Nikodym set N.

How Your Function is Used to Build the Nikodym Set:
Your function directly produces the set N. An external function will then verify if your returned set is a valid Nikodym set by checking the condition for every point x in F_p^3.

Your Goal and Evaluation:
Your goal is to define the set N in such a way that it has the smallest possible size (cardinality) while still being a valid Nikodym set. The size of N is the number of unique points it contains.

Your function will be evaluated based on the negative size of the resulting set N. A smaller set results in a less negative (i.e., higher) score. If your function returns an invalid Nikodym set, it will receive a very large penalty.

Important Hint: You will be evaluated against a range of prime values of p. You must find a general construction that works well for all odd primes. The key to a small set is to make the punctured lines overlap significantly. Think about how you can choose the points in your set N to satisfy the Nikodym property for all p^3 points in the space while using as few points as possible.

Good luck!

Usually we start an experiment with a very simple initial program. In this case, we tried starting it with a more complex program suggested by Gemini. This program did not perform well, and we did not see much difference between these two setups.

In [ ]:
#@title Initial program used

def search_for_best_construction(p: int, d: int):
  """Search for the best Nikodym set for F_p^d."""

  if d == 3:
    # For p >= 5, a smaller construction is the union of the set of points
    # with no zero coordinates and the three coordinate axes.
    # The size is (p-1)^3 + 3p - 2.
    # For p=3, the original construction of size p(p-1)^2 = 12 is better
    # than this construction which would have size (3-1)^3 + 3*3 - 2 = 15.
    if p == 3:
      # For p=3, the set N = {(x,y,z) | z=xy} is a Nikodym set of size p^2=9.
      # This is smaller than the previous p(p-1)^2=12 construction.
      # This construction works because for any point (a,b,c), we can find a
      # line direction v such that the other two points on the line lie on
      # the surface z=xy. This holds specifically for p=3.
      points = []
      for x_coord in range(p):
        for y_coord in range(p):
          z_coord = (x_coord * y_coord) % p
          points.append([x_coord, y_coord, z_coord])
      best_construction = np.array(points, dtype=np.int64)
    elif p >= 5:
      # For p >= 5, a very good construction is the union of points with no zero
      # coordinates and the three coordinate axes. Its size is (p-1)^3 + 3p - 2.
      # This construction for p >= 5 is based on the union of points with no zero
      # coordinates and the three coordinate axes, with a further reduction.
      # The base size is (p-1)^3 + 3p - 2.
      # The "crazy" idea is to improve this by removing specific algebraic curves
      # from the set of points with no zero coordinates, while retaining the
      # Nikodym property.

      # Points with no zero coordinates
      core_points = set(itertools.product(range(1, p), repeat=3))

      # Remove the first twisted cubic C1 = {(t, t^2, t^3) | t in F_p^*}.
      # These points are of the form (t, t^2, t^3).
      # For t in F_p^*, all coordinates are non-zero, so they are in core_points.
      twisted_cubic_1_to_remove = set()
      for t in range(1, p):
        x = t
        y = (t * t) % p
        z = (t * t * t) % p
        twisted_cubic_1_to_remove.add((x, y, z))

      core_points -= twisted_cubic_1_to_remove

      # Remove a second twisted cubic C2 = {(t, t^-1, t^-2) | t in F_p^*}.
      # These points are also in F_p^* for t in F_p^*.
      # This curve intersects C1 only at (1,1,1) because t^2=t^-1 => t^3=1,
      # and t^3=t^-2 => t^5=1. Since gcd(3,5)=1, this implies t^1=1.
      # So, removing C2 reduces the size by (p-1)-1 = p-2 additional points.
      twisted_cubic_2_to_remove = set()
      for t_val in range(1, p):
        t = t_val % p
        t_inv = pow(
            t, p - 2, p
        )  # t^(p-2) mod p is t^-1 mod p by Fermat's Little Theorem
        t_inv_sq = pow(t_inv, 2, p)  # (t^-1)^2 = t^-2 mod p
        x = t
        y = t_inv
        z = t_inv_sq
        twisted_cubic_2_to_remove.add((x, y, z))

      core_points -= twisted_cubic_2_to_remove

      # Points on the axes
      axis_points = set()
      for t in range(p):
        axis_points.add((t, 0, 0))
        axis_points.add((0, t, 0))
        axis_points.add((0, 0, t))

      all_points = list(core_points) + list(axis_points)
      best_construction = np.array(all_points, dtype=np.int64)
    else:  # Should not be reached for odd primes
      # Fallback to original p=3 construction for safety, though it's unused.
      points = []
      for x_coord in range(p):
        for y_coord in range(1, p):
          for z_coord in range(1, p):
            points.append([x_coord, y_coord, z_coord])
      best_construction = np.array(points, dtype=np.int64)

  else:
    # Fallback for other dimensions to prevent errors.
    best_construction = np.array([], dtype=np.int64).reshape(0, d)

  return best_construction


In [ ]:
#@title Code found by AlphaEvolve

"""AlphaEvolve experiment for the Nikodym problem."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from typing import Any, Callable, Mapping, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
from scipy.optimize import milp, LinearConstraint, Bounds
from itertools import product

njit = numba.njit


# Here are the best constructions for small values of the parameter,
# that you have found so far:

# PREVIOUS CONSTRUCTIONS START HERE


best_score_p5_d3_iqhd = -81.99990588967684
normalized_score_p5_d3_iqhd = 1.1714272269953834
best_score_p47_d3_iqhd = -91491.99999796062
normalized_score_p47_d3_iqhd = 0.9394972479869447
best_score_p53_d3_iqhd = -134907.99993632347
normalized_score_p53_d3_iqhd = 0.9590934291871541
best_score_p59_d3_iqhd = -185655.99999514097
normalized_score_p59_d3_iqhd = 0.9512430061440216
best_score_p61_d3_iqhd = -214007.9999261718
normalized_score_p61_d3_iqhd = 0.9904934691253983
best_score_p67_d3_iqhd = -281939.9999098088
normalized_score_p67_d3_iqhd = 0.9804426142
best_score_p71_d3_iqhd = -336143.9999725935
normalized_score_p71_d3_iqhd = 0.9798059881674794
best_score_p73_d3_iqhd = -367899.999911781
normalized_score_p73_d3_iqhd = 0.9854763445813024
best_score_p79_d3_iqhd = -466439.9999355381
normalized_score_p79_d3_iqhd = 0.9827403123589182
best_score_p83_d3_iqhd = -532095.999974577
normalized_score_p83_d3_iqhd = 0.964899936847771
best_score_p89_d3_iqhd = -666519.9999016016
normalized_score_p89_d3_iqhd = 0.9779301074613925


# PREVIOUS CONSTRUCTIONS END HERE


def search_for_best_construction(p: int, d: int):
  """Search for the best Nikodym set for F_p^d."""

  if d == 3:
    # This construction defines a Nikodym set N as F_p^3 \ C, where C is a
    # union of algebraic curves within (F_p^*)^3. The previous complex
    # implementation was equivalent to this simpler set difference.

    # The curves to be removed are of the form (t, t^k, t^j) for t in F_p^*,
    # where the exponent j is "close" to k.
    curves_to_remove = {
        (t, pow(t, k, p), pow(t, (k + diff - 1) % (p - 1) + 1, p))
        for k in range(1, p)
        for diff in [-4, -3, -2, -1, 1, 2, 3, 4]
        for t in range(1, p)
    }

    # Reverting to the correct interpretation of the previous "good" construction:
    # The Nikodym set N is F_p^3 \ C, where C is the set of curves to remove.
    # The goal is to make N as small as possible, which means maximizing |C|.
    all_points = set(itertools.product(range(p), repeat=3))

    # The construction for N is F_p^3 \ C. To make N small, C should be large.
    # The previous construction yielding a good score (for p=5, ~-96) used 'diff'
    # values (shifts in exponents) indicating that 'j' was "close" to 'k'.
    # The current code's broad 'diff' range (all non-zero exponents) resulted in an
    # invalid Nikodym set (score -1000.0). This indicates too many points were removed.
    #
    # This change aims to restore validity by narrowing the range of removed curves,
    # and then to optimize size by carefully choosing the 'M' parameter.
    # 'M' defines the maximum absolute difference between exponents k and j.
    # 'shift' represents the value (j-k) modulo (p-1) for exponents in [0, p-2].
    #
    # We choose M=2 as a starting point. This means for each t,k, we remove points
    # (t, t^k, t^j) where j is k-2, k-1, k+1, or k+2 (mod p-1).
    # This reduces the size of C compared to the 'all_possible_diffs' version,
    # making N larger and hopefully valid. A valid set can then be further optimized.
    M = 4 # This value has historically yielded good performance for similar constructions.
          # It defines the range of shifts for the exponents (j-k) mod (p-1).
          # A value of M=2 was previously tested and led to an invalid Nikodym set,
          # suggesting that too few points were removed from F_p^3, making the
          # Nikodym property difficult to satisfy. Increasing M leads to a larger
          # set C (points removed), and thus a smaller N (Nikodym set).
          # The optimal M should balance validity and minimal size.

    # Generate shift values from -M to M, excluding 0.
    # These represent the difference (j_idx - k_idx) mod (p-1).
    shifts_to_apply = set()
    for s_val in range(-M, M + 1):
        if s_val == 0: # Shifts of 0 mean j=k, these are not to be removed.
            continue
        # Ensure that the shift value is not equivalent to 0 modulo (p-1).
        # This prevents j_idx from becoming equal to k_idx for large M values
        # or small p. (e.g., if M=4, p=5, then -4 and 4 are 0 mod 4).
        if (s_val % (p - 1)) != 0:
            shifts_to_apply.add(s_val)

    # Define the set C of points to be removed from F_p^3.
    # C = { (t, t^k, t^j) | t in F_p^*, k in {1..p-1}, j in {1..p-1} and j != k, where
    # j is related to k by one of the 'shifts_to_apply' values. }
    curves_to_remove = set()
    for t_val in range(1, p): # t in F_p^*
        for k_val in range(1, p): # k in {1..p-1} (exponent for the y-coordinate)
            # Convert k_val to its 0-indexed exponent for modular arithmetic (0 to p-2).
            # e.g., pow(x, exp_val, p) where exp_val in [1, p-1].
            # exp_val 1 -> index 0, exp_val p-1 -> index p-2.
            k_idx = k_val - 1

            for shift_val in shifts_to_apply:
                # Calculate the 0-indexed exponent for j.
                # j_idx must be in [0, p-2].
                j_idx = (k_idx + shift_val) % (p - 1)

                # Convert back to 1-indexed exponent for pow() (1 to p-1).
                j_val = j_idx + 1

                # Add the point to the set of curves to remove.
                point = (t_val, pow(t_val, k_val, p), pow(t_val, j_val, p))
                curves_to_remove.add(point)

    # Add the main diagonal points (t,t,t) for t in F_p^* to the set of points to remove.
    # These points were not included by the previous construction because the 'shifts_to_apply'
    # explicitly excludes shifts of 0 (which would correspond to j=k).
    # Removing these points from N makes N smaller. The Nikodym property is preserved
    # for points (x,x,x) by picking a non-diagonal line direction (e.g., (1,0,0)),
    # as the resulting punctured line points (x+t, x, x) are not in the original
    # 'curves_to_remove' (due to the k=j condition) nor in the newly removed diagonal.
    for t_val in range(1, p):
        curves_to_remove.add((t_val, t_val, t_val))

    # Crazy idea: Additionally remove diagonal lines on coordinate planes.
    # These points are (t,t,0), (t,0,t), (0,t,t) for t in F_p^*.
    # These points have at least one zero coordinate, so they are not included
    # in the 'curves_to_remove' set which requires non-zero coordinates.
    # They are also distinct from (0,0,0) because t is non-zero.
    #
    # If a point x is, e.g., (x_0, x_0, 0) for x_0 != 0, we need to find a line
    # l through x s.t. l \ x is in N.
    # Consider the line l: (x_0, x_0, 0) + t'*(0,0,1) for t' in F_p^*.
    # The points on this line are (x_0, x_0, t') for t' in F_p^*.
    # These points are not in the original 'curves_to_remove' because their z-coordinate
    # is non-zero, and the original set only contained points with (non-zero)^k and (non-zero)^j.
    # They are also not in the new set of removed points (e.g., (x_0, x_0, 0)) as their
    # z-coordinate is not zero.
    # Therefore, (x_0, x_0, t') for t' != 0 are all in N, and the Nikodym property is preserved.
    for t_val in range(1, p):
        curves_to_remove.add((t_val, t_val, 0))
        curves_to_remove.add((t_val, 0, t_val))
        curves_to_remove.add((0, t_val, t_val))

    # The Nikodym set N is F_p^3 \ C.
    all_points = set(itertools.product(range(p), repeat=3))
    nikodym_set_points = all_points - curves_to_remove

    best_construction = np.array(list(nikodym_set_points), dtype=np.int64)
  else:
    # Fallback for other dimensions to prevent errors.
    best_construction = np.array([], dtype=np.int64).reshape(0, d)

  return best_construction


See what Deep Think thought about this construction [here](https://github.com/google-deepmind/alphaevolve_repository_of_problems/blob/main/experiments/finite_field_nikodym_problem/deep_think_docs/nikodym.tex). It claims this set has size around p^3 - 8p^2.

In [ ]:
#@title Evaluation code


@njit
def _is_point_in_construction(
    point_to_check: np.ndarray, construction: np.ndarray
) -> bool:
  """Numba-friendly check if a point exists in the construction array."""
  for i in range(construction.shape[0]):
    is_equal = True
    for j in range(construction.shape[1]):
      if construction[i, j] != point_to_check[j]:
        is_equal = False
        break
    if is_equal:
      return True
  return False


@njit
def is_valid_nikodym_numba(construction: np.ndarray, p: int, d: int) -> bool:
  """Checks if a construction is a valid Nikodym set in F_p^d.

  A Nikodym set N must satisfy the following: for every point x in F_p^d,
  there exists a line l passing through x such that l \ {x} is a subset of N.

  NOTE: This implementation is hardcoded for d=3 to comply with Numba's
  limitation on converting lists to tuples for dynamic tuple creation.
  """
  if d != 3:
    # This function is only valid for d=3.
    return False

  # Create a set from the construction for efficient lookups.
  dummy_tuple = (np.int64(0), np.int64(0), np.int64(0))
  construction_set = {dummy_tuple}
  construction_set.clear()
  for i in range(construction.shape[0]):
    construction_set.add(
        (construction[i, 0], construction[i, 1], construction[i, 2])
    )

  # Helper to check if a "punctured" line is fully contained in the set.
  def check_punctured_line_is_contained(point_x, direction_v, const_set):
    for t in range(1, p):  # We check for t=1, 2, ..., p-1.
      p_arr = (point_x + t * direction_v) % p
      point_on_line = (p_arr[0], p_arr[1], p_arr[2])
      if point_on_line not in const_set:
        return False
    return True

  # --- Main Loop: Iterate through EVERY point x in F_p^3 ---
  for x1 in range(p):
    for x2 in range(p):
      for x3 in range(p):
        x = np.array([x1, x2, x3], dtype=np.int64)
        found_line_for_x = False

        # Search for a direction v for the current point x.
        # Case 1: Directions v = (1, *, *)
        for v2 in range(p):
          if found_line_for_x:
            break
          for v3 in range(p):
            v = np.array([1, v2, v3], dtype=np.int64)
            if check_punctured_line_is_contained(x, v, construction_set):
              found_line_for_x = True
              break
        if found_line_for_x:
          continue

        # Case 2: Directions v = (0, 1, *)
        for v3 in range(p):
          v = np.array([0, 1, v3], dtype=np.int64)
          if check_punctured_line_is_contained(x, v, construction_set):
            found_line_for_x = True
            break
        if found_line_for_x:
          continue

        # Case 3: Direction v = (0, 0, 1)
        v = np.array([0, 0, 1], dtype=np.int64)
        if check_punctured_line_is_contained(x, v, construction_set):
          found_line_for_x = True

        # If after checking all directions, no suitable line was found for x:
        if not found_line_for_x:
          return False  # Not a valid Nikodym set.

  # If we get here, a line was found for every point x.
  return True


def calculate_score(construction: np.ndarray, p: int, d: int) -> float:
  """Calculates the score for a given construction."""
  if (
      not isinstance(construction, np.ndarray)
      or construction.ndim != 2
      or construction.shape[1] != d
  ):
    return np.inf

  if construction.shape[0] == 0:
    return np.inf

  # Remove duplicate points before scoring
  unique_tuples = {tuple(row) for row in construction}
  unique_construction = np.array(list(unique_tuples), dtype=np.int64)
  size = unique_construction.shape[0]

  is_nikodym = is_valid_nikodym_numba(unique_construction, p, d)

  if not is_nikodym:
    return np.inf  # Heavily penalize invalid sets

  return -float(size)


def evaluate(params: int) -> tuple[dict[str, float], dict[str, str]]:
  """Evaluates a Nikodym set construction for given parameters."""
  result = {}
  del params  # Not used directly

  primes_to_test = [5, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89]
  dims_to_test = [3]

  scores = []
  best_constructions = {}
  time_start = time.time()
  for d in dims_to_test:
    for p in primes_to_test:
      construction = search_for_best_construction(p, d)
      score = calculate_score(construction, p, d) + np.random.uniform(0, 0.0001)

      if score != np.inf:
        neg_size = score
        # Normalize score by the size of the best known construction.
        normalized_score = -neg_size / ((p - 1) ** 3 + p + 1)
        if p > 10:
          scores.append(normalized_score)
        logging.info(
            'p=%d, d=%d: |N|=%d, normalized_score (density)=%.4f',
            p,
            d,
            -neg_size,
            normalized_score,
        )

        if p < 3:
          best_constructions[f'best_construction_p{p}_d{d}'] = construction
        best_constructions[f'best_score_p{p}_d{d}'] = neg_size
        best_constructions[f'normalized_score_p{p}_d{d}'] = normalized_score
      else:
        print(f'p={p}, d={d}: Invalid construction found.')
        scores.append(1000)  # Penalize failure

  average_score = (
      np.mean(scores) + np.random.uniform(0, 0.0001) if scores else 1000
  )
  logging.info('Time taken for full evaluation: %s', time.time() - time_start)
  logging.info('Average score (density): %s', average_score)
  result['score'] = -average_score

  return result, best_constructions
